Dans un premier temps, on fait une analyse quantitative pour déterminer à quel niveau les deux retrievers sont complémentaries. On essaie ensuite de déterminer des seuils pour le score qui nous permetteraient de choisir un des deux/en faire un mélange pour incrementer la performance.

## Analyse complémentarité

In [43]:
import json
import numpy as np
import pandas as pd

In [44]:
dense_json = json.load(open("f615_detailed_results.json"))
sparse_json = json.load(open("9abf_detailed_results.json"))

In [45]:
dense_errors = dense_json["errors"]
sparse_errors = sparse_json["errors"]
dense_successes = dense_json["successes"]
sparse_successes = sparse_json["successes"]

In [46]:
sparse_bon = set(sparse_successes.keys())
sparse_mauvais = set(sparse_errors.keys())
dense_bon = set(dense_successes.keys())
dense_mauvais = set(dense_errors.keys())

### Erreurs communes au deux retrievers

Etant donné notre échantillon de réponses correctes et d'erreurs pour le retriever sparse et le retriever dense, vérifions tout d'abord si les deux se trompent souvent sur les memes questions et si oui, en quel pourcentage par rapport aux errors commis au total.

In [32]:
common_errors = list(dense_mauvais.intersection(sparse_mauvais))

In [33]:
print(len(common_errors))
print(len(common_errors)/(len(dense_errors)+len(sparse_errors)))

77
0.2664359861591695


*Dans cet échantillon, 26% des erreurs commis sont commis par les deux retrivers. Il est peut etre intéressant d'aller plus loin dans l'analyse de ces erreurs qui semblent très difficiles à résoudre pour les deux retrievers.*

### Evaluation des cas où le retriever dense fait des erreurs et le sparse trouve la bonne réponse

In [34]:
dense_fails_sparse_good = list(sparse_bon.intersection(dense_mauvais))
print(len(dense_fails_sparse_good))

95


*Pour 95 questions, le retriever sparse donne des bonnes réponses là où le dense se trompe.*

In [35]:
print(len(dense_fails_sparse_good)/len(dense_mauvais))

0.5523255813953488


**Sur les 172 erreurs faites par le retriever dense, dans 55% des cas le sparse peut donner la bonne réponse, et il pourrait donc etre utile de mélanger les deux.**

### Evaluation des cas où le retriever sparse fait des erreurs et le dense trouve la bonne réponse

In [36]:
sparse_fails_dense_good = list(sparse_mauvais.intersection(dense_bon))
print(len(sparse_fails_dense_good))

40


*Pour 40 questions, le retriever dense donne des bonnes réponses là où le sparse se trompe.*

In [37]:
print(len(sparse_fails_dense_good)/len(sparse_mauvais))

0.3418803418803419


**Sur les 117 erreurs faites par le retriever sparse, dans 34% des cas le dense peut donner la bonne réponse.**

### Conclusion:

Le retriever sparse, si mélangé au retriever dense, peut aider à l'améliorer (le sparse donne globalement des meilleures réponses donc c'est assez normal). Dans 34% des cas, le retriever dense peut apporter des bonnes réponses là où le sparse se trompe. Globalement, on ne peut pas parler de complémentarité exacte (dans 26% des cas les deux se trompent et un mélange des deux n'apporteraient donc aucune amélioration), mais coupler le sparse au dense peut augmenter le nombre de bonnes réponses par rapport au dense seul.

## Suite de l'analyse: peut-on trouver un seuil pour 'score' ou 'proba' qui nous indique qu'il faudrait utiliser le retriever dense pour améliorer le sparse?

*exemple de ce qu'on souhaiterait avoir : if [score (dense) < 0.2 et score (sparse) > 0.7 ]: use sparse*

Voici un dataframe qui contient la liste des questions où le retriever dense améliore le sparse, avec les scores 
correspondantes. En particulier:
- colonne 1 *Questions* : questions où le sparse se trompe et le dense donne la bonne réponse
- colonne 2 *k* : valeur (ou valeurs) de k correspondant à la bonne réponse donnée par le dense
- colonne 3 *scores_dense*: scores donnés par le retriever dense pour la question et la valeur de k correspondante
- colonne 4 *best_score_sparse*: meilleur score obtenu pour cette question par le retriever sparse, qui fait une erreur (*attention ici je dois corriger le code car pas de correspondance de lignes dans le csv pour cette colonne*)
- colonne 5 *score_sparse_0_1*: normalisation sur ]0,1[ du score pour le retriever sparse

In [47]:
#colonne des questions
col1 = [sparse_fails_dense_good[k] for k in range(len(sparse_fails_dense_good))] 

#voici une liste qui nous donne les true fiches pour chaque success du sparse

true_fiches_col1 = [dense_successes[sparse_fails_dense_good[k]]['true_fiches'] for k in range(len(col1))]
fiches_pred_dense = [dense_successes[sparse_fails_dense_good[k]]['pred_fiches'] for k in range(len(col1))]

def col_questions(col1,true_fiches_col1,fiches_pred_dense):
    A=[]
    for k in range(len(fiches_pred_dense)):
        for j in range(len(fiches_pred_dense[k])):
            if (str(true_fiches_col1[k][0]) in fiches_pred_dense[k][j][0]):
                A.append(col1[k])
                
    return A

def col_k_dense(true_fiches_col1,fiches_pred_dense):
    M=[]
    for k in range(len(fiches_pred_dense)):
        for j in range(len(fiches_pred_dense[k])):
            if (str(true_fiches_col1[k][0]) in fiches_pred_dense[k][j][0]):
                M.append(fiches_pred_dense[k][j][1])
    return M


def col_scores_k_dense(true_fiches_col1,fiches_pred_dense):
    L=[]
    for k in range(len(fiches_pred_dense)):
        for j in range(len(fiches_pred_dense[k])):
            if (str(true_fiches_col1[k][0]) in fiches_pred_dense[k][j][0]):
                L.append(fiches_pred_dense[k][j][2])
                
    return L




def col_k_dense(true_fiches_col1,fiches_pred_dense):
    M=[]
    for k in range(len(fiches_pred_dense)):
        for j in range(len(fiches_pred_dense[k])):
            if (str(true_fiches_col1[k][0]) in fiches_pred_dense[k][j][0]):
                M.append(fiches_pred_dense[k][j][1])
    return M


fiches_pred_sparse = [sparse_errors[sparse_fails_dense_good[k]]['pred_fiches'] for k in range(len(col1))]

def col_best_score_sparse(fiches_pred_sparse):
    J =[]
    for j in range(len(fiches_pred_sparse)):
        l = [fiches_pred_sparse[j][k][2] for k in range(len(fiches_pred_sparse[j]))]
        J.append(max(l))
    return J


In [48]:
col = col_questions(col1,true_fiches_col1,fiches_pred_dense)
col2 = col_k_dense(true_fiches_col1,fiches_pred_dense)
col3 = col_scores_k_dense(true_fiches_col1,fiches_pred_dense)
col4 = col_best_score_sparse(fiches_pred_sparse) 
col4bis = [(col4[x]-min(col4))/(max(col4)-min(col4))for x in range(len(col4))]

In [50]:
df = pd.DataFrame(list(zip(col,col2,col3,col4,col4bis)), columns =['Questions','k', 'scores_dense','best_score_sparse','score_sparse_0_1'])
df.head(n=10)

,Questions,k,scores_dense,best_score_sparse,score_sparse_0_1
0,Pour Un enfant de 8 ans reconnu par ses parent...,1,0.576475,39.888540,0.600575
1,Pour Un enfant de 8 ans reconnu par ses parent...,2,0.572860,38.415104,0.575890
2,Pour Un enfant de 8 ans reconnu par ses parent...,3,0.560518,25.564970,0.360610
3,Pour Un enfant de 8 ans reconnu par ses parent...,4,0.560518,60.007965,0.937638
4,Pour Un enfant de 8 ans reconnu par ses parent...,5,0.560518,14.889669,0.181766
5,"Bonjour, lorsque j'ai fait faire la carte d'id...",4,0.664292,9.055583,0.084026
6,"En tant que mineur, ai-je le droit d'avoir des...",1,0.655366,27.497390,0.392984
7,"En tant que mineur, ai-je le droit d'avoir des...",2,0.632048,22.873125,0.315513
8,"En tant que mineur, ai-je le droit d'avoir des...",3,0.631823,28.692795,0.413011
9,"Bonjour, Mon conjoint et moi meme nous sommes ...",1,0.563578,38.956085,0.584953


Pb: pas la bonne correspondance entre les questions est le best score pour le retriever sparse, à résoudre

In [42]:
#df.to_csv('erreurs_retriever_sparse_succes_retriever_dense.csv',index=False)

To do:
- résoudre problème au niveau du csv pour le best_score_sparse
- **est-il pertinant de trouver un seuil du score qui relie les deux retrievers? Si oui, comment**